<a href="https://colab.research.google.com/github/SweetDdang/mango/blob/main/mango_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

## 데이터 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/project1

/content/drive/MyDrive/project1


In [ ]:
df = pd.read_csv('mango.csv')

In [ ]:
df = df[['review', 'T_rating']]

In [ ]:
df.columns = ['document',  # 리뷰데이터
              'label'  # 라벨
              ]

In [ ]:
df

,document,label
0,\n 오랜만의 재방문이지만 복수의 리뷰를 이미 작성한 곳이라 자세한...,맛있다
1,\n 나름 가로수길 초입에 위치한 몽리 편안하면서도 이국적인 분위기...,괜찮다
2,\n 와이프가 사랑하는 사람을 생각하고 으이구!!’ 술을 먹더라도 ...,맛있다
3,\n 배달시켜서 먹어봤는ㄷ 그릇이 일반 플라스틱에 담겨 와서 기분이...,괜찮다
4,\n 다른 메뉴들은 <괜찮다> 정도라서 고민했는데 피자가 맛있어서 ...,맛있다
...,...,...
1065,"\n 한줄평: 초심을 잃은건지, 지점마다 다른건지... 실망스러웠던...",별로
1066,\n 한남동 이치류가게로 올라가는 길이 너무 어수선해서맞는 길인가 ...,맛있다
1067,\n 근래 방문했던 한식 컨템퍼러리 레스토랑 중 가장 좋았다재밌는데...,맛있다
1068,\n 한남동 뒷골목에 있는 부토. 이전에 좋아했던 프라이빗 이탈리안...,맛있다


In [ ]:
df['document'] = df['document'].replace('\n','', regex=True)

In [ ]:
df

,document,label
0,오랜만의 재방문이지만 복수의 리뷰를 이미 작성한 곳이라 자세한 설...,맛있다
1,나름 가로수길 초입에 위치한 몽리 편안하면서도 이국적인 분위기로 ...,괜찮다
2,와이프가 사랑하는 사람을 생각하고 으이구!!’ 술을 먹더라도 안주...,맛있다
3,배달시켜서 먹어봤는ㄷ 그릇이 일반 플라스틱에 담겨 와서 기분이 그...,괜찮다
4,다른 메뉴들은 <괜찮다> 정도라서 고민했는데 피자가 맛있어서 (사...,맛있다
...,...,...
1065,"한줄평: 초심을 잃은건지, 지점마다 다른건지... 실망스러웠던 소...",별로
1066,한남동 이치류가게로 올라가는 길이 너무 어수선해서맞는 길인가 생각...,맛있다
1067,근래 방문했던 한식 컨템퍼러리 레스토랑 중 가장 좋았다재밌는데 맛...,맛있다
1068,한남동 뒷골목에 있는 부토. 이전에 좋아했던 프라이빗 이탈리안 가...,맛있다


In [ ]:
df['label'] = df['label'].replace(['맛있다'], 1) # 긍정/부정 라벨링 
df['label'] = df['label'].replace(['괜찮다'], 1)
df['label'] = df['label'].replace(['별로'], 0)

## train, test split

In [ ]:
train,test = train_test_split(df, test_size=0.1, shuffle=True, random_state=34) # random_state=34 고정

In [ ]:
print(train.shape)
print(test.shape)

(963, 2)
(107, 2)


## 전처리 훈련셋

In [ ]:
# 리뷰 문장 추출
sentences = train['document']

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

In [ ]:
# 라벨 추출
labels = train['label'].values
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
train['label'].value_counts()

1    889
0     74
Name: label, dtype: int64

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

[CLS] 
          블랙트러플크림파케리와 프로슈토와루꼴라 먹음.파스타와 피자 모두 만족.재방문의사 있음.
         [SEP]
['[CLS]', '블', '##랙', '##트', '##러', '##플', '##크', '##림', '##파', '##케', '##리', '##와', '프로', '##슈', '##토', '##와', '##루', '##꼴', '##라', '먹', '##음', '.', '파', '##스', '##타', '##와', '피', '##자', '모두', '만', '##족', '.', '재', '##방', '##문', '##의', '##사', '있', '##음', '.', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9379, 118958,  12605,  30873,  21614,   8896,  18784,
        12030,   9291,  20308,  12605,  14646, 119265,    198,   9738,
        21155,   9486,  21928,  12030,  28911,   9069, 118963,   9254,
        38696,  11925,    198,   9309,  16439,  12692,   9901,  12605,
        22695,  11018,   9641,  52951,  11925,  13764,  26344,   9682,
         9981,  22333,  23665,  19105,   9069,  18227,  22440,  79633,
         9657,  22333, 118615,  11903,    198,   9615,  85297,  37712,
        27023,  12092,   9781,  58931,  35506,  14040,  11664,    198,
        19709,   9272, 118758,   9266, 119185,  30873,   9144,   8843,
        11664, 119088,  11903,    102,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9638, 118644,  10739,   8924, 118864,  48387, 118864,   9344,
         15184,  37004,  11925,   9495, 119138, 119081,  48345,    102,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## 전처리 - 테스트셋


In [ ]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

860               비밀스러운 공간인 몽크스부쳐~채식 식당인데 대박 맛집이다~미나리 ...
313               분위기도 좋고 맛있는 곳!내방역 근처파스타도 타다끼도 맛있고 친절...
134               생면 파스타의 면 자체에 좀 더 집중할 수 있다. 계란 노른자를 ...
757               매주1회 방문하는곳입니다..월.화는 휴무이고요 . 봉골레파스타 좋...
561               감베리크레마는 생각보다 많이 매콤해서 오히려 라자냐가 제입맛에는 ...
1009                                       맛있습니다.        
629                                     맛잇거 친절햇어요        
269               근래 방문했던 한식 컨템퍼러리 레스토랑 중 가장 좋았다재밌는데 맛...
402               가성비를 어디서 찾아야하는건지..?파인다이닝을 톡톡밖에 안가봐서 ...
112               토요일 런치 2명예약 후 방문 가능한 오스테리아오르조 애프터8원래...
Name: document, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:30]

['[CLS]           비밀스러운 공간인 몽크스부쳐~채식 식당인데 대박 맛집이다~미나리 파스타는 일품이다자리는 좀 협소하지만 대화하기 좋은 장소같다~웨이터분들도 친절하시고~다른 메뉴 먹으러 또 가고싶다         [SEP]',
 '[CLS]           분위기도 좋고 맛있는 곳!내방역 근처파스타도 타다끼도 맛있고 친절하시다         [SEP]',
 '[CLS]           생면 파스타의 면 자체에 좀 더 집중할 수 있다. 계란 노른자를 섞어 먹으면 된다. 치즈와 계란 노른자, 트러플의 향이 은은하게 면을 감싼다. 쏘스 맛 자체가 그리 강하진 않다. 면에 최대한 집중할 수 있다. 쏘스 맛이 강하진 않지만 충분히 꾸덕하고 짭짤해서 맛있다. 얇고 납작한 생면의 탱글한 식감이 온전히 잘 느껴진다. 집 가는 길에 또 은근히 생각나게 하는 맛이다. 바질페스토 파스타도 너무나 먹고 싶었는데 타야린 자체만으로도 충분히 헤비해서 못 먹었다. 다시 와 보고 싶은 파스타 집이다.         [SEP]',
 '[CLS]           매주1회 방문하는곳입니다..월.화는 휴무이고요 . 봉골레파스타 좋아하시면 꼭 먹어봐야해요. 봉골레맛집입니다!!!! 봉골레 잘하는곳 흔치않아요 . 다른것들도 맛있지만, 봉골레파스타는 따라올 가게가 없어요         [SEP]',
 '[CLS]           감베리크레마는 생각보다 많이 매콤해서 오히려 라자냐가 제입맛에는 잘맞았음. 그래도 맛있는집!         [SEP]',
 '[CLS]           맛있습니다.         [SEP]',
 '[CLS]           맛잇거 친절햇어요         [SEP]',
 '[CLS]           근래 방문했던 한식 컨템퍼러리 레스토랑 중 가장 좋았다재밌는데 맛있기까지 맛있는 곳이야 워낙 많지만 재미까지 있다니 다한 기분음식 외에 매장의 절반을 차지하는 키친이 상당히 인상적이다 게다가 완전이 오픈되어 있어 눈높이가 맞아 시종일관 요리를 하는 모습을 테

In [ ]:
# 라벨 추출
labels = test['label'].values
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 
          비밀스러운 공간인 몽크스부쳐~채식 식당인데 대박 맛집이다~미나리 파스타는 일품이다자리는 좀 협소하지만 대화하기 좋은 장소같다~웨이터분들도 친절하시고~다른 메뉴 먹으러 또 가고싶다
         [SEP]
['[CLS]', '비', '##밀', '##스', '##러', '##운', '공', '##간', '##인', '몽', '##크', '##스', '##부', '##쳐', '~', '채', '##식', '식', '##당', '##인', '##데', '대', '##박', '맛', '##집', '##이다', '~', '미', '##나', '##리', '파', '##스', '##타', '##는', '일', '##품', '##이다', '##자', '##리는', '좀', '협', '##소', '##하지', '##만', '대', '##화', '##하기', '좋은', '장', '##소', '##같', '##다', '~', '웨', '##이터', '##분', '##들', '##도', '친', '##절', '##하', '##시', '##고', '~', '다른', '메', '##뉴', '먹', '##으', '##러', '또', '가', '##고', '##싶', '##다', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9379, 118958,  12605,  30873,  21614,   8896,  18784,
        12030,   9291,  20308,  12605,  14646, 119265,    198,   9738,
        21155,   9486,  21928,  12030,  28911,   9069, 118963,   9254,
        38696,  11925,    198,   9309,  16439,  12692,   9901,  12605,
        22695,  11018,   9641,  52951,  11925,  13764,  26344,   9682,
         9981,  22333,  23665,  19105,   9069,  18227,  22440,  79633,
         9657,  22333, 118615,  11903,    198,   9615,  85297,  37712,
        27023,  12092,   9781,  58931,  35506,  14040,  11664,    198,
        19709,   9272, 118758,   9266, 119185,  30873,   9144,   8843,
        11664, 119088,  11903,    102,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9379, 118958,  12605,  30873,  21614,   8896,  18784,  12030,
          9291,  20308,  12605,  14646, 119265,    198,   9738,  21155,   9486,
         21928,  12030,  28911,   9069, 118963,   9254,  38696,  11925,    198,
          9309,  16439,  12692,   9901,  12605,  22695,  11018,   9641,  52951,
         11925,  13764,  26344,   9682,   9981,  22333,  23665,  19105,   9069,
         18227,  22440,  79633,   9657,  22333, 118615,  11903,    198,   9615,
         85297,  37712,  27023,  12092,   9781,  58931,  35506,  14040,  11664,
           198,  19709,   9272, 118758,   9266, 119185,  30873,   9144,   8843,
         11664, 119088,  11903,    102,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## 모델 생성

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

SystemError: ignored

In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

No GPU available, using the CPU instead.


In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

RuntimeError: ignored

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

## 모델 학습

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.31
  Training epcoh took: 0:17:52

Running Validation...
  Accuracy: 0.91
  Validation took: 0:00:37

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.27
  Training epcoh took: 0:17:51

Running Validation...
  Accuracy: 0.91
  Validation took: 0:00:37

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.18
  Training epcoh took: 0:17:52

Running Validation...
  Accuracy: 0.91
  Validation took: 0:00:37

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.12
  Training epcoh took: 0:17:55

Running Validation...
  Accuracy: 0.94
  Validation took: 0:00:37

Training complete!


## 테스트셋 평가

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.93
Test took: 0:00:41


## 새로운 문장 테스트

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
logits = test_sentences(['생긴건 별로지만 맛 하나는 끝내줌!'])

print(logits)
print(np.argmax(logits))

[[-1.8519435  2.1826222]]
1


In [ ]:
logits = test_sentences([''])

print(logits)
print(np.argmax(logits))

[[-1.9661278  2.2066424]]
1
